<a href="https://colab.research.google.com/github/brielle5810/TJS_Project/blob/brielle-test/colabVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import cv2
import numpy as np
!pip install pytesseract
!pip install keras-ocr
import pytesseract
import os
import PIL
from PIL import Image, ImageEnhance
from pip._internal.metadata import pkg_resources
!pip install symspellpy
from symspellpy import SymSpell, Verbosity

#import keras_ocr
#import tensorflow as tf

#pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

uncropped_path = 'uncropped/'
cropped_path = 'cropped/'


In [33]:
!pip install tesseract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tesseract: filename=tesseract-0.1.3-py3-none-any.whl size=45562552 sha256=3d6c10ff9067242d0f11471908f3aaa8c1a85aede21fe79002359942e90caeb6
  Stored in directory: /root/.cache/pip/wheels/b4/47/6e/bb7543eee5b12cf0bbeedd33b40886429a79aef0b03d76e051
Successfully built tesseract


In [34]:
! apt install tesseract-ocr
! apt install libtesseract-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (8,376 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [35]:
!which tesseract

/usr/bin/tesseract


In [36]:
import pytesseract

import cv2
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
import keras_ocr
import tensorflow as tf

pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [20]:
!pip show pytesseract

Name: pytesseract
Version: 0.3.13
Summary: Python-tesseract is a python wrapper for Google's Tesseract-OCR
Home-page: https://github.com/madmaze/pytesseract
Author: Samuel Hoffstaetter
Author-email: samuel@hoffstaetter.com
License: Apache License 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: packaging, Pillow
Required-by: 


In [15]:

def crop_left_half(image_path):
    after_path = 'cut_turned/'
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    width = image.shape[1]

    fraction = 2 / 5
    new_width = int(width * fraction)
    left_half = image[:, :new_width]

    cv2.imwrite(after_path + image_path.split('/')[1], left_half)

def rotate_180(image_path):
    after_path = 'rotated/'
    image = Image.open(image_path)
    image = image.rotate(180)
    image.save(after_path + image_path.split('/')[1])
    crop_left_half(after_path + image_path.split('/')[1])

def preprocess(image_path):
    img = cv2.imread(image_path, 0)
    ret, imgf = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY, cv2.THRESH_OTSU)

    imgf = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    cv2.imwrite('preprocessed/' + image_path.split('/')[1], imgf)


def binarize(image_path):

    image = cv2.imread(image_path)

    # normalize image
    image = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    # noise removal
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel, iterations=2)
    opening = cv2.fastNlMeansDenoisingColored(opening, None, 10, 10, 7, 15)

    gray = cv2.cvtColor(opening, cv2.COLOR_BGR2GRAY)
    gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    # thinning, skeletonization
    kernel = np.ones((3, 3), np.uint8) # 5, 5 -> thicker idk
    erosion = cv2.erode(gray, kernel, iterations=1)
    #dilation = cv2.dilate(erosion, kernel, iterations=1)
    #show image
    cv2.imwrite('preprocessed/' + image_path.split('/')[1], erosion)

def extract_text(image_path):
    image = cv2.imread(image_path)
    text = pytesseract.image_to_string(image, lang='eng', config='--psm 4')
    #image_to_data(im, lang='eng', config=psm)
    return text

def extract_text_keras(image_path): #KERAS BLOWS
    pipeline = keras_ocr.pipeline.Pipeline()
    images = [keras_ocr.tools.read(image_path)]
    prediction_groups = pipeline.recognize(images)
    text = ''
    for predictions in prediction_groups:
        for text in predictions:
            text += text
    return text


In [45]:

def post_correction(text):
    sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
    dictionary_path = "frequency_dictionary_en_82_765.txt"

    dictionary_path2 = "newDict.txt"
    sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1, separator="$")
    #print(text)

    sym_spell.load_dictionary(dictionary_path2, term_index=0, count_index=1, separator="$")

    # print first 10 words in dictionary


    #suggestions = sym_spell.lookup_compound(text, max_edit_distance=2, transfer_casing=True, ignore_non_words=True)
    suggestions = sym_spell.lookup(
        text, Verbosity.CLOSEST, max_edit_distance=2, include_unknown=True
    )

    return suggestions

In [38]:
# !which pytesseract


# !pip list

In [46]:

    # for file in os.listdir('uncropped/'):
    #     rotate_180(f'uncropped/{file}')

    # for file in os.listdir('cut_turned/'):
    #     binarize(f'cut_turned/{file}')

    #count1 = 0
    text1 = ''
    for file in os.listdir('preprocessed/'):
            text1 = extract_text(f'preprocessed/{file}')

            corrections = post_correction(text1)
            for suggestion in corrections:
                print(suggestion.term)
            count1 += 1


    # text2 = ''
    # count2 = 0
    # for file in os.listdir('preprocessed/'):
    #     while count2 < 1:
    #         text2 = extract_text(f'preprocessed/{file}')
    #         count2 += 1
    #print(text1)
    #print(text2)

Phoebis sehnac
maycellina

MGCL 1169103

USA: Arizona
Graham Co.
Mt. Graham
10,000 ft.
3-VII-1972
B. Weber

Allyn Museum Acc.
1981-24 «
B. Weber

 

Phoebis sennae

maycellina

ARIZ: Santa Cruz Ce.
Sycamore Canyon
2-IxX-77
Teg. 8. O’ Hara

 

Phoebis sehnae

maycellina

MGCL 1167602

P. es. marcellina
June 28, 1969
Brownsville, Tex.
Cameron Co.

leg Heitzman

 

Phoebis sennae

maycellina

Pei, Sullnue

Bentsen-Hio Grande |:
State Park
Hidalgo Co., Tex.

 

